In [3]:
import numpy as np
from astropy.io import fits
from astropy.table import Table

import sys
sys.path.append('/Users/nityaravi/Documents/Github/RotationCurves/spirals/')
from dark_matter_mass_v1 import rot_fit_BB

In [59]:
HESS_FOLDER = '/Users/nityaravi/Documents/Research/data/manga/DRP_map_Hessians/'
tab_fn = '/Users/nityaravi/Documents/Research/data/manga/output_files/final2table.txt'
tab2_fn = '/Users/nityaravi/Documents/Research/data/manga/output_files/DRP_HaVel_map_results_BB_smooth_lt_2.0_.fits'

In [60]:
tab = Table.read(tab_fn, format='ascii.commented_header')
tab[0]

plateifu,Mtot,Mtot_err,Mstar,Mstar_err,MHI,MH2,Mdust,Mr,Z,R90,v_max,r_turn,alpha
str11,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
10001-3702,11.052790322774381,10.928525167884018,10.0200393746564,12.86078304577374,8.947139739990234,8.77132441413365,6.942434416615007,-19.4337,8.754925012287872,13.263102531433105,313.6762095221658,0.7925717678215113,8.720845050623494


In [19]:
tab2 = Table.read(tab2_fn)
tab2[0]

plate,ifudsgn,plateifu,mangaid,versdrp2,versdrp3,verscore,versutil,versprim,platetyp,srvymode,objra,objdec,ifuglon,ifuglat,ifura,ifudec,ebvgal,nexp,exptime,drp3qual,bluesn2,redsn2,harname,frlplug,cartid,designid,cenra,cendec,airmsmin,airmsmed,airmsmax,seemin,seemed,seemax,transmin,transmed,transmax,mjdmin,mjdmed,mjdmax,gfwhm,rfwhm,ifwhm,zfwhm,mngtarg1,mngtarg2,mngtarg3,catidnum,plttarg,manga_tileid,nsa_iauname,ifudesignsize,ifutargetsize,ifudesignwrongsize,z,zmin,zmax,szmin,szmax,ezmin,ezmax,probs,pweight,psweight,psrweight,sweight,srweight,eweight,esweight,esrweight,nsa_field,nsa_run,nsa_camcol,nsa_version,nsa_nsaid,nsa_nsaid_v1b,nsa_z,nsa_zdist,nsa_sersic_absmag [7],nsa_elpetro_absmag [7],nsa_elpetro_amivar [7],nsa_sersic_mass,nsa_elpetro_mass,nsa_elpetro_ba,nsa_elpetro_phi,nsa_extinction [7],nsa_elpetro_th50_r,nsa_petro_th50,nsa_petro_flux [7],nsa_petro_flux_ivar [7],nsa_elpetro_flux [7],nsa_elpetro_flux_ivar [7],nsa_sersic_ba,nsa_sersic_n,nsa_sersic_phi,nsa_sersic_th50,nsa_sersic_flux [7],nsa_sersic_flux_ivar [7],smoothness_score,nsa_elpetro_th90,v_sys,v_sys_err,ba,ba_err,x0,x0_err,y0,y0_err,phi,phi_err,v_max,v_max_err,r_turn,r_turn_err,chi2,alpha,alpha_err,Rmax,M,M_err,fit_flag
int64,bytes32,bytes32,bytes32,bytes32,bytes32,bytes32,bytes32,bytes32,bytes32,bytes32,float64,float64,float64,float64,float64,float64,float64,int64,float64,int64,float64,float64,bytes53,int64,bytes32,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,int64,int64,int64,int64,bytes32,int64,bytes19,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,bytes6,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
10001,12701,10001-12701,1-48157,v3_1_1,v3_1_1,v1_9_1,v5_5_36,v2_9,APOGEE-2&MaNGA,MaNGA dither,133.371090612,57.5984251446,159.27921887,38.8381829181,133.37109,57.598425,0.0545561,9,8100.81,0,20.0835,41.292,ma079,22,3,9114,134.41357,57.604214,1.10715,1.15321,1.21404,1.78219,1.85994,2.12685,0.898721,0.929654,0.937466,57372,57373,57373,2.69906,2.64546,2.59645,2.54039,2080,0,0,1,plateTargets-1.par,6013,J085329.03+573554.9,127,127,0,0.0392074,0.0193562,0.0264775,0.0386959,0.0429176,0.0193562,0.0264775,0.748562,-999.0,0.665321,0.705378,1.08789,1.19925,-999.0,0.66562,0.705713,203,1350,6,v1_0_1,51226,-999,0.0392074,0.0395926,-16.2982 .. -19.5112,-16.2321 .. -19.5522,275.541 .. 452.944,3068010000.0,2685340000.0,0.417013,69.2463,0.448783 .. 0.0800664,5.16633,3.63987,12.7947 .. 413.049,3.26768 .. 0.000422625,13.8306 .. 425.941,3.03694 .. 0.00496838,0.335333,0.741845,68.2584,5.29184,14.8124 .. 412.19,2.50888 .. 0.0150605,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--


In [22]:
H_0 = 100      # Hubble's Constant in units of h km/s/Mpc
c = 299792.458 # Speed of light in units of km/s

In [43]:
v_R90 = np.zeros(88)
v_R90_err = np.zeros(88)

In [45]:
for i in range (0,88):
    
    gal_ID = tab['plateifu'][i]
    i_params = np.where(tab['plateifu'][i] == tab2['plateifu'])[0][0]
    z = tab2['z'][i_params]
    params = [tab2['v_max'][i_params], tab2['r_turn'][i_params], tab2['alpha'][i_params]]
    r90 = tab2['nsa_elpetro_th90'][i_params]
    
    ############################################################################
    # Convert r90 from arcsec to kpc
    #---------------------------------------------------------------------------
    dist_to_galaxy_Mpc = c*z/H_0
    dist_to_galaxy_kpc = dist_to_galaxy_Mpc*1000

    r90_kpc = dist_to_galaxy_kpc*np.tan(r90*(1./60)*(1./60)*(np.pi/180))
    ############################################################################

    ############################################################################
    # Calculate velocity at given radius
    #---------------------------------------------------------------------------
    hess = np.load(HESS_FOLDER + gal_ID + '_Hessian.npy')

    N_samples = 10000

    v_samples = np.zeros(N_samples, dtype=float)

    v = rot_fit_BB(r90_kpc, params) # params is the set of best-fit values for the model
    v_R90[i] = v

    try:

        param_samples = np.random.multivariate_normal(mean=params, cov=0.5*hess[-3:,-3:], size=N_samples)

        for j in range(N_samples):
                
            if np.all(param_samples[j] > 0):
                v_samples[j] = rot_fit_BB(r90_kpc, param_samples[j])

        v_err = np.std(v_samples[np.isfinite(v_samples)])
        v_R90_err[i] = v_err
    
    except np.linalg.LinAlgError:
        v_err = np.nan
        v_R90_err[i] = v_err
    ############################################################################

/Users/nityaravi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: covariance is not positive-semidefinite.


In [61]:
col1 = Table.Column(name='v_R90', data=v_R90)
col2 = Table.Column(name='v_R90_err', data=v_R90_err)

In [62]:
tab.add_column(col1)

In [63]:
tab

plateifu,Mtot,Mtot_err,Mstar,Mstar_err,MHI,MH2,Mdust,Mr,Z,R90,v_max,r_turn,alpha,v_R90
str11,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
10001-3702,11.052790322774381,10.928525167884018,10.0200393746564,12.86078304577374,8.947139739990234,8.77132441413365,6.942434416615007,-19.4337,8.754925012287872,13.263102531433105,313.6762095221658,0.7925717678215113,8.720845050623494,313.6762052707942
7815-3702,10.613039578007609,10.294796645844741,10.441004906748342,13.435080247636705,9.002732276916504,9.006900445142412,6.963697702470532,-20.0207,8.644526172790677,10.38451099395752,199.46555714049532,1.1936356887733697,7.930411988698445,199.46479810036843
7990-6104,10.651277732422022,10.366704314336037,10.156365239029698,12.511487931293033,9.559786796569824,9.463747580850782,7.350859799653533,-19.84,8.526241889021023,16.161773681640625,213.16018677919567,1.2679737743976607,1.0024670875178068,176.94340907589887
7992-6104,10.679068928886512,10.931951002910834,10.352457455190857,11.777541918384703,9.142339706420898,9.080653272468302,7.083779295020389,-19.9142,8.656355276096175,17.880041122436523,370.25235829482443,4.406683328587263,0.700065575499098,170.68607758793047
8077-12704,10.867338335470095,10.636390301650888,9.99319696409062,12.58406390041193,9.619999885559082,8.493991621794368,7.227032576663279,-19.7518,8.565284957451594,23.467266082763672,210.46636915420416,0.8222675298899436,1.0585089143440773,194.70416198773384
8077-6102,10.659860699154235,10.355629999068695,9.816905376165465,10.829595243833525,9.309956550598145,8.744420323140734,7.00784554464874,-18.655,8.582427528583263,16.799543380737305,317.70041105671606,8.626072047704287,3.9822430490810157,191.15485862982027
8080-12702,11.178796131677775,10.251123830832475,10.869727403183214,11.977514016153917,10.25059700012207,9.61940129961914,8.035643218275176,-20.4291,8.679040247387787,42.67671203613281,197.64676312533655,5.343154170164197,34.34946918375905,197.64676312533655
8082-12702,11.614500335079871,11.577018357671777,11.015169113136754,14.608237508686512,10.02690315246582,9.245969289935173,7.894666832091676,-21.6454,8.781368352259319,44.79909896850586,323.50053283634924,0.7099201517535662,21.72672932085675,323.5005328363492
8146-1901,10.542383187889719,10.16183658209533,10.25663327898968,13.266177780012208,9.15999984741211,9.014225690297076,7.160611658939073,-20.0406,8.748315021302002,7.916838645935059,219.58841715125416,0.9766459800774372,5.056868853419022,219.46511239680967


In [64]:
tab.add_column(col2)

In [65]:
tab

plateifu,Mtot,Mtot_err,Mstar,Mstar_err,MHI,MH2,Mdust,Mr,Z,R90,v_max,r_turn,alpha,v_R90,v_R90_err
str11,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
10001-3702,11.052790322774381,10.928525167884018,10.0200393746564,12.86078304577374,8.947139739990234,8.77132441413365,6.942434416615007,-19.4337,8.754925012287872,13.263102531433105,313.6762095221658,0.7925717678215113,8.720845050623494,313.6762052707942,3.204736962978333
7815-3702,10.613039578007609,10.294796645844741,10.441004906748342,13.435080247636705,9.002732276916504,9.006900445142412,6.963697702470532,-20.0207,8.644526172790677,10.38451099395752,199.46555714049532,1.1936356887733697,7.930411988698445,199.46479810036843,0.03103360940014056
7990-6104,10.651277732422022,10.366704314336037,10.156365239029698,12.511487931293033,9.559786796569824,9.463747580850782,7.350859799653533,-19.84,8.526241889021023,16.161773681640625,213.16018677919567,1.2679737743976607,1.0024670875178068,176.94340907589887,nan
7992-6104,10.679068928886512,10.931951002910834,10.352457455190857,11.777541918384703,9.142339706420898,9.080653272468302,7.083779295020389,-19.9142,8.656355276096175,17.880041122436523,370.25235829482443,4.406683328587263,0.700065575499098,170.68607758793047,5.179308209073311
8077-12704,10.867338335470095,10.636390301650888,9.99319696409062,12.58406390041193,9.619999885559082,8.493991621794368,7.227032576663279,-19.7518,8.565284957451594,23.467266082763672,210.46636915420416,0.8222675298899436,1.0585089143440773,194.70416198773384,14.610365765242049
8077-6102,10.659860699154235,10.355629999068695,9.816905376165465,10.829595243833525,9.309956550598145,8.744420323140734,7.00784554464874,-18.655,8.582427528583263,16.799543380737305,317.70041105671606,8.626072047704287,3.9822430490810157,191.15485862982027,0.3663906501000165
8080-12702,11.178796131677775,10.251123830832475,10.869727403183214,11.977514016153917,10.25059700012207,9.61940129961914,8.035643218275176,-20.4291,8.679040247387787,42.67671203613281,197.64676312533655,5.343154170164197,34.34946918375905,197.64676312533655,0.05178676539332151
8082-12702,11.614500335079871,11.577018357671777,11.015169113136754,14.608237508686512,10.02690315246582,9.245969289935173,7.894666832091676,-21.6454,8.781368352259319,44.79909896850586,323.50053283634924,0.7099201517535662,21.72672932085675,323.5005328363492,nan
8146-1901,10.542383187889719,10.16183658209533,10.25663327898968,13.266177780012208,9.15999984741211,9.014225690297076,7.160611658939073,-20.0406,8.748315021302002,7.916838645935059,219.58841715125416,0.9766459800774372,5.056868853419022,219.46511239680967,0.0


In [66]:
tab.write('/Users/nityaravi/Documents/Research/data/manga/output_files/final3table.txt',
                    format='ascii.commented_header',
                    overwrite=True)